In [1]:
import os 
import numpy as np
import tensorflow as tf

from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, Convolution2D, core, BatchNormalization
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.layers.core import Lambda
import keras

from keras import backend as K
from keras.backend import binary_crossentropy

# config = tf.ConfigProto()
# config.gpu_options.allocator_type ='BFC'
# config.gpu_options.per_process_gpu_memory_fraction = 0.90

Using TensorFlow backend.


In [2]:
%run 'data.ipynb'

------------------------------
Creating training images...
------------------------------
306
loading done
Saving to .npy files done.
------------------------------
Creating test images...
------------------------------
77
loading done
Saving to imgs_test.npy files done.


In [3]:
def slice_batch(x, n_gpus, part):
    """
    Divide the input batch into [n_gpus] slices, and obtain slice no.
    [part].

    i.e. if len(x)=10, then slice_batch(x, 2, 1) will return x[5:].

    """

    sh = K.shape(x)

    L = sh[0] / n_gpus
    L = tf.cast(L, tf.int32)

    if part == n_gpus - 1:

        return x[part*L:]

    return x[part*L:int(part+1)*L]

def to_multi_gpu(model, n_gpus=4):

    """Given a keras [model], return an equivalent model which parallelizes

    the computation over [n_gpus] GPUs.



    Each GPU gets a slice of the input batch, applies the model on that
    slice

    and later the outputs of the models are concatenated to a single
    tensor,

    hence the user sees a model that behaves the same as the original.

    """

    with tf.device('/cpu:0'):

        x = Input(model.input_shape[1:], name=model.input_names[0])


    towers = []

    for g in range(n_gpus):

        with tf.device('/gpu:' + str(g)):

            slice_g = Lambda(slice_batch, lambda shape: shape, arguments={'n_gpus':n_gpus, 'part':g})(x)

            towers.append(model(slice_g))


    with tf.device('/cpu:0'):

        merged = merge(towers, mode='concat', concat_axis=0)

    new_model = Model(input=[x], output=merged)
    
    funcType = type(model.save)
    # monkeypatch the save to save just the underlying model
    def new_save(self_,filepath, overwrite=True):
        model.save(filepath, overwrite)
    new_model.save=funcType(new_save, new_model)
   
    return new_model

In [4]:
smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + binary_crossentropy(y_pred, y_true)


In [5]:
class myUnet(object):

    def __init__(self, tp = 'buildings', img_rows = 512, img_cols = 512):

        self.type = tp
        self.img_rows = img_rows
        self.img_cols = img_cols

    def load_data(self):

        mydata = dataProcess(self.img_rows, self.img_cols)
        imgs_train, imgs_mask_train = mydata.load_train_data(self.type)
        imgs_test = mydata.load_test_data()
        return imgs_train, imgs_mask_train, imgs_test

    #Define the neural network
    def get_unet(self):
        inputs = Input((512, 512, 1))
        #
        conv1 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(inputs)
        conv1 = BatchNormalization(mode=0, axis=1)(conv1)
        conv1 = keras.layers.advanced_activations.ELU()(conv1)
        conv1 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv1)
        conv1 = BatchNormalization(mode=0, axis=1)(conv1)
        conv1 = keras.layers.advanced_activations.ELU()(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        #
        conv2 = Convolution2D(64, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(pool1)
        conv2 = BatchNormalization(mode=0, axis=1)(conv2)
        conv2 = keras.layers.advanced_activations.ELU()(conv2)
        conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same', kernel_initializer = 'he_normal')(conv2)
        conv2 = BatchNormalization(mode=0, axis=1)(conv2)
        conv2 = keras.layers.advanced_activations.ELU()(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        #
        conv3 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(pool2)
        conv3 = BatchNormalization(mode=0, axis=1)(conv3)
        conv3 = keras.layers.advanced_activations.ELU()(conv3)
        conv3 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv3)
        conv3 = BatchNormalization(mode=0, axis=1)(conv3)
        conv3 = keras.layers.advanced_activations.ELU()(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        #
        conv4 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(pool3)
        conv4 = BatchNormalization(mode=0, axis=1)(conv4)
        conv4 = keras.layers.advanced_activations.ELU()(conv4)
        conv4 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv4)
        conv4 = BatchNormalization(mode=0, axis=1)(conv4)
        conv4 = keras.layers.advanced_activations.ELU()(conv4)
#         drop4 = Dropout(0.5)(conv?4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
        #
        conv5 = Convolution2D(512, 3, 3, border_mode='same', kernel_initializer='he_normal')(pool4)
        conv5 = BatchNormalization(mode=0, axis=1)(conv5)
        conv5 = keras.layers.advanced_activations.ELU()(conv5)
        conv5 = Convolution2D(512, 3, 3, border_mode='same', kernel_initializer='he_normal')(conv5)
        conv5 = BatchNormalization(mode=0, axis=1)(conv5)
        conv5 = keras.layers.advanced_activations.ELU()(conv5)
#         drop5 = Dropout(0.5)(conv5)
        #
        up1 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=-1)
        conv6 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(up1)
        conv6 = BatchNormalization(mode=0, axis=1)(conv6)
        conv6 = keras.layers.advanced_activations.ELU()(conv6)
        conv6 = Convolution2D(256, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv6)
        conv6 = BatchNormalization(mode=0, axis=1)(conv6)
        conv6 = keras.layers.advanced_activations.ELU()(conv6)
        #
        up2 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=-1)
        conv7 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(up2)
        conv7 = BatchNormalization(mode=0, axis=1)(conv7)
        conv7 = keras.layers.advanced_activations.ELU()(conv7)
        conv7 = Convolution2D(128, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv7)
        conv7 = BatchNormalization(mode=0, axis=1)(conv7)
        conv7 = keras.layers.advanced_activations.ELU()(conv7)
        #
        up3 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=-1)
        conv8 = Convolution2D(64, 3, 3, border_mode='same', kernel_initializer='he_normal')(up3)
        conv8 = BatchNormalization(mode=0, axis=1)(conv8)
        conv8 = keras.layers.advanced_activations.ELU()(conv8)
        conv8 = Convolution2D(64, 3, 3, border_mode='same', kernel_initializer='he_normal')(conv8)
        conv8 = BatchNormalization(mode=0, axis=1)(conv8)
        conv8 = keras.layers.advanced_activations.ELU()(conv8)
        #
        up4 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=-1)
        conv9 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(up4)
        conv9 = BatchNormalization(mode=0, axis=1)(conv9)
        conv9 = keras.layers.advanced_activations.ELU()(conv9)
        conv9 = Convolution2D(32, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv9)
        conv9 = BatchNormalization(mode=0, axis=1)(conv9)
        conv9 = keras.layers.advanced_activations.ELU()(conv9)
        conv9 = Convolution2D(2, 3, 3, border_mode='same', kernel_initializer = 'he_normal')(conv9)
        #
        conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

        model = Model(input=inputs, output=conv10)

        model.compile(optimizer= Nadam(lr = 1e-3), loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])
        model = to_multi_gpu(model)
        model.compile(optimizer= Nadam(lr = 1e-3), loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])

        return model


    def train(self):

        print("loading data")
        imgs_train, imgs_mask_train, imgs_test = self.load_data()
        print("loading data done")
        model = self.get_unet()
        print("got unet")

        model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss',verbose=1, save_best_only=True, mode='min')
        print('Fitting model...')
#         model.fit(imgs_train, imgs_mask_train, batch_size=8, nb_epoch=10, verbose=1, validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])
        model.fit(imgs_train, imgs_mask_train, batch_size=8, nb_epoch=50, verbose=1, validation_split=0.3, shuffle=True, callbacks=[model_checkpoint])

        
        print('predict test data')
        imgs_mask_test = model.predict(imgs_test, batch_size=8, verbose=1)
        if self.type == 'buildings':
            np.save('../results/imgs_mask_test.npy', imgs_mask_test)
        elif self.type == 'cars':
            np.save('../results/imgs_mask_test_cars.npy', imgs_mask_test)

    def save_img(self):

        print("array to image")
        if self.tp == 'buildings':
            imgs = np.load('../results/imgs_mask_test.npy')
        elif self.tp == 'cars':
            imgs = np.load('../results/imgs_mask_test_cars.npy')
        for i in range(imgs.shape[0]):
            img = imgs[i]
            img = array_to_img(img)
            img.save("../results/%d.jpg"%(i))

In [6]:
myunet = myUnet('buildings')
myunet.train()

loading data
------------------------------
load train images...
------------------------------
------------------------------
load test images...
------------------------------
loading data done


/usr/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `BatchNormalization` call to the Kera

got unet
Fitting model...
Train on 214 samples, validate on 92 samples
Epoch 1/50
214/214 [==============================] - 65s 303ms/step - loss: 5.4608 - binary_crossentropy: 0.5198 - jaccard_coef_int: 0.1002 - val_loss: 9.7585 - val_binary_crossentropy: 1.7026 - val_jaccard_coef_int: 0.0224
Epoch 2/50
214/214 [==============================] - 41s 190ms/step - loss: 4.0478 - binary_crossentropy: 0.6757 - jaccard_coef_int: 0.2007 - val_loss: 5.9909 - val_binary_crossentropy: 1.1746 - val_jaccard_coef_int: 0.0414
Epoch 3/50
214/214 [==============================] - 40s 187ms/step - loss: 3.9438 - binary_crossentropy: 0.8064 - jaccard_coef_int: 0.2079 - val_loss: 3.6266 - val_binary_crossentropy: 0.7250 - val_jaccard_coef_int: 0.1948
Epoch 4/50
214/214 [==============================] - 41s 192ms/step - loss: 3.8934 - binary_crossentropy: 0.8803 - jaccard_coef_int: 0.2157 - val_loss: 3.3808 - val_binary_crossentropy: 0.7257 - val_jaccard_coef_int: 0.2458
Epoch 5/50
214/214 [=========

214/214 [==============================] - 41s 192ms/step - loss: 2.4965 - binary_crossentropy: 0.9670 - jaccard_coef_int: 0.4249 - val_loss: 2.0477 - val_binary_crossentropy: 0.7159 - val_jaccard_coef_int: 0.4530
Epoch 21/50
214/214 [==============================] - 41s 191ms/step - loss: 2.5292 - binary_crossentropy: 0.9834 - jaccard_coef_int: 0.4177 - val_loss: 2.2434 - val_binary_crossentropy: 0.7012 - val_jaccard_coef_int: 0.3989
Epoch 22/50
214/214 [==============================] - 41s 191ms/step - loss: 2.3694 - binary_crossentropy: 0.9640 - jaccard_coef_int: 0.4517 - val_loss: 2.1638 - val_binary_crossentropy: 0.7433 - val_jaccard_coef_int: 0.4243
Epoch 23/50
214/214 [==============================] - 41s 193ms/step - loss: 2.7021 - binary_crossentropy: 1.0989 - jaccard_coef_int: 0.3958 - val_loss: 8.2181 - val_binary_crossentropy: 2.4910 - val_jaccard_coef_int: 0.1889
Epoch 24/50
214/214 [==============================] - 40s 189ms/step - loss: 2.5482 - binary_crossentropy: 

214/214 [==============================] - 41s 192ms/step - loss: 1.6277 - binary_crossentropy: 0.7859 - jaccard_coef_int: 0.5974 - val_loss: 1.5950 - val_binary_crossentropy: 0.6447 - val_jaccard_coef_int: 0.5469
Epoch 41/50
214/214 [==============================] - 41s 191ms/step - loss: 1.6244 - binary_crossentropy: 0.7720 - jaccard_coef_int: 0.5883 - val_loss: 1.6521 - val_binary_crossentropy: 0.7390 - val_jaccard_coef_int: 0.5584
Epoch 42/50
214/214 [==============================] - 41s 191ms/step - loss: 1.5586 - binary_crossentropy: 0.7462 - jaccard_coef_int: 0.6063 - val_loss: 1.7353 - val_binary_crossentropy: 0.7488 - val_jaccard_coef_int: 0.5133
Epoch 43/50
214/214 [==============================] - 41s 190ms/step - loss: 1.6771 - binary_crossentropy: 0.8140 - jaccard_coef_int: 0.5830 - val_loss: 1.6166 - val_binary_crossentropy: 0.7658 - val_jaccard_coef_int: 0.5471
Epoch 44/50
214/214 [==============================] - 41s 192ms/step - loss: 1.5184 - binary_crossentropy: 

In [7]:
myunet = myUnet('cars')
myunet.train()
# myunet.save_img()

loading data
------------------------------
load train images...
------------------------------
------------------------------
load test images...
------------------------------
loading data done


/usr/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal")`
/usr/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `BatchNormalization` call to the Kera

got unet
Fitting model...


/usr/lib/python3.5/site-packages/ipykernel_launcher.py:116: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 214 samples, validate on 92 samples
Epoch 1/50
214/214 [==============================] - 64s 300ms/step - loss: 6.0951 - binary_crossentropy: 0.1637 - jaccard_coef_int: 0.0056 - val_loss: 6.6437 - val_binary_crossentropy: 0.1324 - val_jaccard_coef_int: 0.0962
Epoch 2/50
214/214 [==============================] - 43s 200ms/step - loss: 4.4629 - binary_crossentropy: 0.0952 - jaccard_coef_int: 0.0404 - val_loss: 5.6077 - val_binary_crossentropy: 0.1749 - val_jaccard_coef_int: 0.0980
Epoch 3/50
214/214 [==============================] - 43s 199ms/step - loss: 2.4076 - binary_crossentropy: 0.1102 - jaccard_coef_int: 0.1548 - val_loss: 2.6077 - val_binary_crossentropy: 0.1525 - val_jaccard_coef_int: 0.1797
Epoch 4/50
214/214 [==============================] - 41s 191ms/step - loss: 2.0915 - binary_crossentropy: 0.1218 - jaccard_coef_int: 0.1870 - val_loss: 2.1911 - val_binary_crossentropy: 0.1299 - val_jaccard_coef_int: 0.2303
Epoch 5/50
214/214 [==============================] - 4

214/214 [==============================] - 43s 199ms/step - loss: 1.5791 - binary_crossentropy: 0.1911 - jaccard_coef_int: 0.2865 - val_loss: 1.6778 - val_binary_crossentropy: 0.2003 - val_jaccard_coef_int: 0.3210
Epoch 22/50
214/214 [==============================] - 43s 200ms/step - loss: 1.4819 - binary_crossentropy: 0.1771 - jaccard_coef_int: 0.3096 - val_loss: 1.6405 - val_binary_crossentropy: 0.2013 - val_jaccard_coef_int: 0.3238
Epoch 23/50
214/214 [==============================] - 42s 197ms/step - loss: 1.4770 - binary_crossentropy: 0.1828 - jaccard_coef_int: 0.3108 - val_loss: 1.7202 - val_binary_crossentropy: 0.2076 - val_jaccard_coef_int: 0.3127
Epoch 24/50
214/214 [==============================] - 43s 202ms/step - loss: 1.4214 - binary_crossentropy: 0.1777 - jaccard_coef_int: 0.3220 - val_loss: 1.5483 - val_binary_crossentropy: 0.2036 - val_jaccard_coef_int: 0.3500
Epoch 25/50
214/214 [==============================] - 43s 200ms/step - loss: 1.3518 - binary_crossentropy: 

214/214 [==============================] - 42s 198ms/step - loss: 1.1753 - binary_crossentropy: 0.1613 - jaccard_coef_int: 0.3949 - val_loss: 1.3795 - val_binary_crossentropy: 0.1839 - val_jaccard_coef_int: 0.4040
Epoch 42/50
214/214 [==============================] - 42s 198ms/step - loss: 1.2976 - binary_crossentropy: 0.1792 - jaccard_coef_int: 0.3662 - val_loss: 1.3892 - val_binary_crossentropy: 0.1824 - val_jaccard_coef_int: 0.4094
Epoch 43/50
214/214 [==============================] - 43s 201ms/step - loss: 1.1549 - binary_crossentropy: 0.1618 - jaccard_coef_int: 0.4010 - val_loss: 1.3485 - val_binary_crossentropy: 0.1784 - val_jaccard_coef_int: 0.4186
Epoch 44/50
214/214 [==============================] - 43s 199ms/step - loss: 1.1782 - binary_crossentropy: 0.1668 - jaccard_coef_int: 0.3957 - val_loss: 1.3858 - val_binary_crossentropy: 0.1917 - val_jaccard_coef_int: 0.3936
Epoch 45/50
214/214 [==============================] - 42s 196ms/step - loss: 1.1495 - binary_crossentropy: 

In [ ]:
# # define the checkpoint
# filepath="results/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

# # fit the model
# model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

# # load the network weights
# filename = "results/weights-improvement-49-0.4748-bigger.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='adam')